In [1]:
from natsort import natsorted
from models import *
# import tensorflow as tf
import torch
import torch.nn as nn
import random 
import numpy as np
import os
import scipy.io
import sent2vec
import re
from scipy.spatial.distance import cdist
from sklearn.metrics import accuracy_score
import pickle
import numpy as np

In [2]:
labels = np.loadtxt("kinetics_600_labels.csv",dtype=str,delimiter =',')
Kin_classes = list(labels[:,1])
Kin_classes.pop(0)

'name'

In [3]:
old_labels = list(np.load("folders.npy"))

In [4]:
predicted_embed = np.load("ucf_predictions_oe.npy")
class_embed = np.load("ucf_class_embeddings.npy")
true_embed = np.load("ucf_true_embeddings.npy")

In [5]:
def compute_accuracy(predicted_embed, class_embed, true_embed):
    """
    Compute accuracy based on the closest Word2Vec class
    """
    assert len(predicted_embed) == len(true_embed), "True and predicted labels must have the same number of samples"
    y_pred = cdist(predicted_embed, class_embed, 'cosine').argsort(1)
    y = cdist(true_embed, class_embed, 'cosine').argmin(1)
    accuracy = accuracy_score(y, y_pred[:, 0]) * 100
    accuracy_top5 = np.mean([l in p for l, p in zip(y, y_pred[:, :5])]) * 100
    return accuracy, accuracy_top5, y_pred[:, 0]

In [6]:
accuracy, accuracy_top5, out_pre = compute_accuracy(predicted_embed, class_embed, true_embed)

In [7]:
true_label = np.loadtxt("test.csv",dtype=str)[:,1].astype(int)

In [8]:
accuracy_split, accuracy_split_top5 = np.zeros(10), np.zeros(10)
for split in range(10):
    # Select test set
    np.random.seed(12) # fix seed for future comparability
    sel_classes = np.random.permutation(len(class_embed))[:len(class_embed) // 2]
    sel = [l in sel_classes for l in true_label]
    test_classes = len(sel_classes)

    # Compute accuracy
    subclasses = np.unique(true_label[sel])
    tl = np.array([int(np.where(l == subclasses)[0]) for l in true_label[sel]])
    acc, acc5,yy = compute_accuracy(predicted_embed[sel], class_embed[sel_classes], true_embed[sel])
    accuracy_split[split] = acc
    accuracy_split_top5[split] = acc5

In [9]:
np.mean(accuracy_split)

67.61919315403425